# Q-Learning in Reinforcement Learning 

Q-Learning is a Reinforcement learning policy that will find the next best action, given a current state. It chooses this action at random and aims to maximize the reward.

Q-learning is a model-free, off-policy reinforcement learning that will find the best course of action, given the current state of the agent. Depending on where the agent is in the environment, it will decide the next action to be taken. 

Important Terms in Q-Learning

1. States: The State, S, represents the current position of an agent in an environment. 
2. Action: The Action, A, is the step taken by the agent when it is in a particular state.
3. Rewards: For every action, the agent will get a positive or negative reward.
4. Episodes: When an agent ends up in a terminating state and can’t take a new action.
5. Q-Values: Used to determine how good an Action, A, taken at a particular state, S, is. Q (A, S).
6. Temporal Difference: A formula used to find the Q-Value by using the value of current state and action and previous state and action.

In Q-Learning, the agent has an initial Q-Table with arbitary fixed values (could be zero). 

Initally, the agent will be in start position in the environment denoted as ($s_{t=0}$)


At each time($t$), the agent 
- Performs an Action ($a_t$)
- Observe the reward ($r_t$)
- Enter to a new state ($s_{t+1}$)

Then, based on operation the Q-Table updates.


For updating the Q-table the algorithm uses the **Bellman Equation** which states

$$Q(s_t, a_t) = R(s_t, a_t) + \gamma * Max(Q(s_{t+1}, A)) $$

where, 
- $s_t$ is current state
- $a_t$ is current action 
- $R(s_t, a_t)$ is reward for current action and state
- $\gamma$ is discount factor
- $s_{t+1}$ is the next state
- $A$ is all actions
- $Q(s_{t+1}, A)$ is the q-value for all action in next state

In [1]:
import gym
import numpy as np
import random
from IPython.display import clear_output

For understanding the Q-Learning technique, we are using Frozen Lake environment from OpenAI Gym. <br>
To know more about [FrozenLake-v1](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/)


![Image](https://www.gymlibrary.dev/_images/frozen_lake.gif "Frozen Lake Without Slippery")

In [2]:
# Defining the Environment Name
env_name = 'FrozenLake-v1'

In [3]:
# Environment Utility and Initialization class

class Environment(gym.wrappers.time_limit.TimeLimit):
    
    def __init__(self, env_name, *args, **kwargs):
        super(Environment, self).__init__(gym.make(env_name,  *args, **kwargs))
        self.print_env()
    
    def print_env(self):
        print("Environment Name: ", self.spec.name)
        print("Action Space Type: ", "DISCRETE" if type(self.action_space) == gym.spaces.discrete.Discrete else "CONTINUOUS" )
        print("Observation Space Type: ", "DISCRETE" if type(self.observation_space) == gym.spaces.discrete.Discrete else "CONTINUOUS" )
        print("Observation Space: ", self.observation_space)
        
    def __del__(self):
        self.close()

In [4]:
env = Environment(env_name, render_mode="rgb_array", is_slippery=False, map_name="8x8")

Environment Name:  FrozenLake
Action Space Type:  DISCRETE
Observation Space Type:  DISCRETE
Observation Space:  Discrete(64)


# Agent 

In this notebook, we are extending the Generic agent, we created in previous [notebook](https://github.com/ritikjain51/open-AI-GYM/blob/master/Introduction%20to%20GYM.ipynb).


In [5]:
class GenericAgent:
    
    def __init__(self, env: gym.wrappers.time_limit.TimeLimit):
        action_space = env.action_space

        # Action Space
        if type(action_space) == gym.spaces.discrete.Discrete:
            self.action_size = action_space.n
            self.action_type = "DISCRETE"
            self.action_space = np.arange(action_space.n)
        else:
            self.action_type = "CONTINUOUS"
            self.action_low = action_space.low
            self.action_high = action_space.high
            self.action_size = action_space.shape
        
        
        # Observation Type
        obs_space = env.observation_space
        if type(obs_space) == gym.spaces.discrete.Discrete:
            self.observation_type = "DISCRETE"
            self.observation_size = obs_space.n
        else:
            self.observation_type = "CONTINUOUS"
            self.observation_low = obs_space.low
            self.observation_high = obs_space.high
            self.observation_size = obs_space.shape
        
    
    def get_random_action(self, state = None):
        
        if self.action_type == "DISCRETE":
            return np.random.choice(self.action_space)
        else:
            return np.random.uniform(low=self.action_low, high=self.action_high, size=self.action_shape)

## Q-Agent 

I am defining the Q-Agent, that will find the next best action based on current observations. 

### Algorithm 

1. Define the Q-Table with action and observation space. Initialize it to zeros.
2. Select the best action for the observation from Q-Table
    $$a = Max(Q(O_t, A)) $$
    where, 
    - $O_t$ is the current observation
    - $A$ is the all action
    
3. Update the Q-Table based on selection
    $$Q(s_t, a_t) = R(s_t, a_t) + \gamma * Max(Q(s_{t+1}, A)) $$


In [6]:
class QAgent(GenericAgent):
    
    def __init__(self, env: gym.wrappers.time_limit.TimeLimit, learning_rate = 0.57, discount_rate = 0.97, epsilon=1):
        
        super(QAgent, self).__init__(env)
        self.env = env
        self.initalize_qtable()
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.epsilon = epsilon
        self.explore, self.exploit = 0, 0
    
    def initalize_qtable(self):
        # Creating the Q Table with almost zero values
        self.q_table = 1e-4 * np.random.random(size=[self.observation_size, self.action_size])
#         self.q_table = np.zeros([self.observation_size, self.action_size])
    
    def get_action(self, observation):
        
        if random.random() < self.epsilon:
            self.explore += 1
            return self.get_random_action(observation)
        state_table = self.q_table[observation]
        action = np.argmax(state_table) 
        self.exploit += 1
        return action

    def update_qtable(self, experience, action):
        
        observation, reward, terminate, truncate, info = experience
        q_next = self.q_table[observation]
        q_next = np.zeros([self.action_size]) if terminate else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        q_update = q_target - self.q_table[observation, action]
        self.q_table[observation, action] += self.learning_rate * q_update
        
        
        if terminate:
            self.epsilon *= 0.99

In [8]:
agent = QAgent(env)

# Training for 100 episodes
prev_obs = env.observation_space.sample()
total_reward = 0

In [9]:
from time import sleep

In [ ]:

for episode in range(1000): 
    env.reset()
    agent.explore = 0
    agent.exploit = 0
    terminate = False
    sleep(0.1)
    while not terminate:
        env.render()
        action = agent.get_action(prev_obs)
        
        (observe, reward, terminate, truncate, info) = env.step(action)
        total_reward += reward
        agent.update_qtable((observe, reward, terminate, truncate, info), action)
        print("Episode: {}, Reward: {}".format(episode, total_reward))
        print("Previous Observation: {}, Observation: {}".format(prev_obs, observe))
        print("Action: {} Epsilon: {}".format(action, agent.epsilon))
        print("Exploration Step: {} Exploitation Steps: {}\n\n".format(agent.explore, agent.exploit))
        if terminate:
            env.reset()
        
        prev_obs = observe
        print(agent.q_table)
        clear_output(wait=True)

Episode: 88, Reward: 0.0
Previous Observation: 9, Observation: 10
Action: 2 Epsilon: 0.41294967113388825
Exploration Step: 18 Exploitation Steps: 23


[[4.06414750e-05 2.93022612e-05 4.18984278e-05 4.06414750e-05]
 [7.45364592e-05 7.68417105e-05 7.45364592e-05 7.45364592e-05]
 [8.10461794e-05 8.35527622e-05 8.10461794e-05 8.10461794e-05]
 [4.77890489e-05 3.17499628e-05 4.63604754e-05 4.64204563e-05]
 [8.12947199e-05 8.41642601e-05 8.16388291e-05 8.16392777e-05]
 [7.35879477e-05 7.76562281e-05 7.52641610e-05 6.52797829e-05]
 [8.58103747e-05 8.82415639e-05 8.89048462e-05 8.67884436e-05]
 [7.09754722e-05 2.96850339e-05 6.86775717e-05 6.79505953e-05]
 [2.52921831e-05 2.53320236e-05 6.29782113e-07 2.60675085e-05]
 [4.70580495e-05 4.70602750e-05 4.85134469e-05 4.71241876e-05]
 [4.90235948e-05 4.75551357e-05 4.75528870e-05 4.75650635e-05]
 [7.02217767e-05 7.17325778e-05 6.95806008e-05 6.68519364e-06]
 [7.37543265e-05 7.15429305e-05 7.15416982e-05 7.16716996e-05]
 [8.07811001e-05 8.39971450e-0

In [26]:
env.close()

In [16]:
experience

(5, 0.0, True, False, {'prob': 0.3333333333333333})

In [17]:
experience[2]

True